In [36]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [35]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

org.h2.jdbc.JdbcSQLException:  Table "PESSOA" already exists; SQL statement

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;
SELECT * FROM UsoMedicamento


# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [37]:
SELECT Medicamento.NomeMedicamento, COUNT(*) QuantidadeDeUsuarios,UsoMedicamento.CodMedicamento
FROM Medicamento, UsoMedicamento
WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
GROUP BY Medicamento.NomeMedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [38]:
DROP VIEW IF EXISTS MedicamentosUsados;

CREATE VIEW MedicamentosUsados AS 
SELECT Medicamento.NomeMedicamento, UsoMedicamento.CodMedicamento, COUNT(Medicamento.NomeMedicamento) QuantidadeUsuarios
FROM Medicamento, UsoMedicamento
WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
GROUP BY  Medicamento.NomeMedicamento;

SELECT CodMedicamento  FROM MedicamentosUsados
WHERE QuantidadeUsuarios = (SELECT MAX(QuantidadeUsuarios) FROM MedicamentosUsados)
GROUP BY CodMedicamento;



d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [39]:
SELECT Medicamento.NomeMedicamento, COUNT(*) QuantidadeUsuarios
FROM Medicamento, UsoMedicamento
WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
GROUP BY Medicamento.NomeMedicamento
ORDER BY QuantidadeUsuarios DESC
LIMIT 1;



## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [40]:
SELECT UsoMedicamento.CodMedicamento, COUNT(*) QuantidadeUsuarios
FROM Medicamento, UsoMedicamento
WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
GROUP BY UsoMedicamento.CodMedicamento
ORDER BY QuantidadeUsuarios DESC
LIMIT 1;


## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [41]:
  
SELECT AVG(TotalMedicamentos) Media
FROM (SELECT COUNT(*) TotalMedicamentos
      FROM UsoMedicamento
      GROUP BY IdPessoa);


3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [42]:
DROP VIEW IF EXISTS UsoPorUsuario;

CREATE VIEW UsoPorUsuario AS
SELECT IdPessoa, COUNT(*) TotalMedicamentos
      FROM UsoMedicamento
      GROUP BY IdPessoa;

SELECT IdPessoa FROM UsoPorUsuario
WHERE TotalMedicamentos > (SELECT AVG(TotalMedicamentos) Media
                             FROM (SELECT COUNT(*) TotalMedicamentos
                                   FROM UsoMedicamento
                                   GROUP BY IdPessoa));

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [43]:
DROP VIEW IF EXISTS UsoPorUsuario;

CREATE VIEW UsoPorUsuario AS 
SELECT Pessoa.Id, Pessoa.Idade, COUNT(Pessoa.Id) Uso
FROM Pessoa, UsoMedicamento
WHERE Pessoa.Id = UsoMedicamento.IdPessoa
GROUP BY Pessoa.Id;

SELECT Idade, AVG(Uso)  FROM UsoPorUsuario
GROUP BY Idade
ORDER BY Idade DESC;


# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [44]:
DROP VIEW IF EXISTS ConjuntoDeMedicamentos;

CREATE VIEW ConjuntoDeMedicamentos AS
SELECT U1.CodMedicamento medicamentoA, U2.CodMedicamento medicamentoB
FROM UsoMedicamento U1, UsoMedicamento U2
WHERE U1.CodMedicamento > U2.CodMedicamento AND U1.IdPessoa = U2.IdPessoa;

SELECT * FROM ConjuntoDeMedicamentos;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [45]:
SELECT medicamentoA, medicamentoB, COUNT(*) QntConjunto FROM ConjuntoDeMedicamentos
GROUP BY medicamentoA, medicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [47]:
DROP TABLE IF EXISTS Matriz;

CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [48]:
SELECT * FROM Matriz;

UPDATE Matriz
SET IdadeAte60 = TRUE
WHERE Idade <= 60;

SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [49]:

UPDATE Matriz
SET MedicamentosAcimaMedia = TRUE
WHERE 1 <= (SELECT COUNT(IdPessoa) 
            FROM UsoMedicamento
            WHERE Id = IdPessoa AND UsoMedicamento.diasUso > 365 
            GROUP BY IdPessoa);
            
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [55]:
UPDATE Matriz
SET MedicamentosAcimaMedia = TRUE
WHERE ((SELECT COUNT(IdPessoa) 
       FROM UsoMedicamento
       WHERE Matriz.Id = UsoMedicamento.IdPessoa
       GROUP BY IdPessoa) > (SELECT AVG(TotalMedicamentos) Media
                             FROM (SELECT COUNT(*) TotalMedicamentos
                                   FROM UsoMedicamento
                                   GROUP BY IdPessoa)));

SELECT * FROM Matriz


## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

##### Podemos analisar se a maioria das pessoas com mais de 60 anos usam remédios acima da média.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [67]:
DROP VIEW IF EXISTS Idoso;

CREATE VIEW Idoso AS 
SELECT * FROM Matriz
WHERE Idade >= 60;

CREATE VIEW TotalIdoso AS 
SELECT COUNT(*) TotalIdoso FROM Idoso;

CREATE VIEW MedAcimaMedia AS 
SELECT COUNT(MedicamentosAcimaMedia) MedicamentoAcimaMedia FROM Idoso
WHERE MedicamentosAcimaMedia = True;

SELECT MedicamentoAcimaMedia MA, TotalIdoso



1464